In [1]:
from helpers import *
from models import Tiny
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pyrealsense2 as rs
import torch

In [ ]:
N_CLASSES = 6
IMG_SIZE = (416, 416)

paths = {"img": "data/rifle.jpg", 
         "names": "data/custom.names",
         "weights": "data/custom.weights"}
names = load_names(paths["names"])

model = Tiny(N_CLASSES, IMG_SIZE)
model.load_weights(paths["weights"])

if torch.cuda.is_available():
    print("Using CUDA ...")
    model.cuda()

x = read_img(paths["img"])
x = to_yolo_input(x)

@timeit
def detect(model, x):
    detections = model(x)
    detections = nms(detections, 
                     confidence_threshold=0.05, 
                     overlap_threshold=0.5)
    return detections

detections = detect(model, x)

img = read_img(paths["img"])
img = cv2.resize(img, (416, 416))
img = np.array(img)

plt.figure()
fig, ax = plt.subplots()
ax.imshow(img)

cmap = plt.get_cmap("rainbow")
colors = [cmap(i) for i in np.linspace(0, 1, N_CLASSES)]

for bboxes in detections:
    for x1, y1, x2, y2, confidence, probability, class_prediction in bboxes:
        class_idx = int(class_prediction)
        width, height = x2 - x1, y2 - y1
        color = colors[class_idx]
        patch = patches.Rectangle((x1, y1), 
                                  width, height, 
                                  linewidth=2, 
                                  edgecolor=color, 
                                  facecolor="none")
        ax.add_patch(patch)
        plt.text(x1, y1, 
                 s=names[int(class_prediction)], 
                 color="white",  
                 bbox={"color": color})
        print("Label: {}\tObject Confidence: {:.5f}\tClass Probability: {:.5f}"
              .format(names[class_idx], confidence, probability))